In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pretrained/distilbert_step_15000.pt


In [2]:
!pip install wandb

In [3]:
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import os
import nltk
from tqdm.auto import tqdm
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wan = user_secrets.get_secret("wandb_api")
wandb.login(key=wan)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mihika-usgaonker (mihika-usgaonker-none). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
"""
from datasets import load_dataset

# Load TinyStories dataset
#dataset = load_dataset("roneneldan/TinyStories")

# Print dataset structure
#print(dataset)
#print(dataset["train"][0])  # Print the first sample
"""

In [ ]:
"""
from transformers import AutoTokenizer

MODEL_CHECKPOINT = "distilbert-base-uncased"  # Using DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Tokenize dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Convert to PyTorch tensors
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Print a sample
print(tokenized_datasets["train"][0])
"""

In [ ]:
"""
import torch
def mask_tokens(inputs, mlm_probability=0.15):
    # Only print for the first batch processed
    if not hasattr(mask_tokens, "printed_first_batch"):
        # Print original input before masking (numerical form)
        print("Original Input IDs (first sequence):")
        print(inputs[0].tolist())
        
        labels = inputs.clone()
        # We sample a few tokens in each sequence for masked-LM training (with probability `mlm_probability`)
        probability_matrix = torch.full(labels.shape, mlm_probability)
        special_tokens_mask = [
            tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
        ]
        probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100  # We only compute loss on masked tokens

        # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

        # 10% of the time, we replace masked input tokens with random word
        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(0, tokenizer.vocab_size, labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]

        # The rest of the time (10% of the time) we keep the original token
        
        # Print masked input after masking (numerical form)
        print("Masked Input IDs (first sequence):")
        print(inputs[0].tolist())
        
        # Print which tokens were masked (showing only the first sequence)
        print("Masked positions (first sequence):")
        first_seq_masked = masked_indices[0].tolist()
        masked_positions = [i for i, mask in enumerate(first_seq_masked) if mask]
        print(f"Positions: {masked_positions}")
        
        # Print labels for the first sequence
        print("Labels for first sequence (original token IDs at masked positions, -100 elsewhere):")
        print(labels[0].tolist())
        
        # Set the flag to avoid printing for subsequent batches
        mask_tokens.printed_first_batch = True
        
        return inputs, labels
    else:
        # For all other batches, just do the masking without printing
        labels = inputs.clone()
        probability_matrix = torch.full(labels.shape, mlm_probability)
        special_tokens_mask = [
            tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
        ]
        probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100  # We only compute loss on masked tokens

        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(0, tokenizer.vocab_size, labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]

        return inputs, labels

# Apply masking to the dataset
def mask_dataset(examples):
    inputs, labels = mask_tokens(examples["input_ids"])
    return {"input_ids": inputs, "labels": labels, "attention_mask": examples["attention_mask"]}

masked_datasets = tokenized_datasets.map(mask_dataset, batched=True)
"""

In [ ]:
#wandb.init(project="not-distilbert-pretraining", name="not-distilbert-summarization")

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_size, num_attention_heads, dropout_rate):
        super().__init__()
        self.num_attention_heads = num_attention_heads  # Critical bug fix
        self.attention_head_size = hidden_size // num_attention_heads
        self.all_head_size = self.num_attention_heads * self.attention_head_size
        
        self.query = nn.Linear(hidden_size, self.all_head_size)
        self.key = nn.Linear(hidden_size, self.all_head_size)
        self.value = nn.Linear(hidden_size, self.all_head_size)
        
        self.output = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        
    def transpose_for_scores(self, x):
        batch_size, seq_length = x.size(0), x.size(1)
        x = x.view(batch_size, seq_length, self.num_attention_heads, self.attention_head_size)
        return x.permute(0, 2, 1, 3)
    
    def forward(self, hidden_states, attention_mask=None):
        batch_size, seq_length = hidden_states.size(0), hidden_states.size(1)
        
        query = self.query(hidden_states)
        key = self.key(hidden_states)
        value = self.value(hidden_states)
        
        query = self.transpose_for_scores(query)
        key = self.transpose_for_scores(key)
        value = self.transpose_for_scores(value)
        
        attention_scores = torch.matmul(query, key.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        
        if attention_mask is not None:
            attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
            attention_mask = (1.0 - attention_mask) * -10000.0
            attention_scores = attention_scores + attention_mask
        
        attention_probs = F.softmax(attention_scores, dim=-1)
        attention_probs = self.dropout(attention_probs)
        
        context = torch.matmul(attention_probs, value)
        context = context.permute(0, 2, 1, 3).contiguous()
        context = context.view(batch_size, seq_length, self.all_head_size)
        
        output = self.output(context)
        
        return output

class FeedForward(nn.Module):
    def __init__(self, hidden_size, intermediate_size, dropout_rate):
        super().__init__()
        self.dense1 = nn.Linear(hidden_size, intermediate_size)
        self.dense2 = nn.Linear(intermediate_size, hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, hidden_states):
        hidden_states = self.dense1(hidden_states)
        hidden_states = F.gelu(hidden_states)
        hidden_states = self.dense2(hidden_states)
        hidden_states = self.dropout(hidden_states)
        return hidden_states

class TransformerLayer(nn.Module):
    def __init__(self, hidden_size, num_attention_heads, intermediate_size, dropout_rate):
        super().__init__()
        self.attention = MultiHeadAttention(hidden_size, num_attention_heads, dropout_rate)
        self.feed_forward = FeedForward(hidden_size, intermediate_size, dropout_rate)
        self.layer_norm1 = nn.LayerNorm(hidden_size, eps=1e-12)
        self.layer_norm2 = nn.LayerNorm(hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, hidden_states, attention_mask=None):
        attention_output = self.attention(hidden_states, attention_mask)
        attention_output = self.dropout(attention_output)
        hidden_states = self.layer_norm1(hidden_states + attention_output)
        
        ff_output = self.feed_forward(hidden_states)
        hidden_states = self.layer_norm2(hidden_states + ff_output)
        
        return hidden_states


In [ ]:
class DistilBertModel(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers, num_attention_heads, 
                 intermediate_size, max_position_embeddings, dropout_rate):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, hidden_size)
        self.position_embeddings = nn.Embedding(max_position_embeddings, hidden_size)
        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(dropout_rate)
        
        self.layers = nn.ModuleList([
            TransformerLayer(hidden_size, num_attention_heads, intermediate_size, dropout_rate)
            for _ in range(num_layers)
        ])
        
        self.mlm_head = nn.Linear(hidden_size, vocab_size)
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def forward(self, input_ids, attention_mask=None, labels=None):
        batch_size, seq_length = input_ids.size()
        
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand(batch_size, -1)
        
        inputs_embeds = self.embeddings(input_ids)
        position_embeds = self.position_embeddings(position_ids)
        
        embeddings = inputs_embeds + position_embeds
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        
        hidden_states = embeddings
        for layer in self.layers:
            hidden_states = layer(hidden_states, attention_mask)
        
        prediction_scores = self.mlm_head(hidden_states)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            masked_lm_loss = loss_fct(prediction_scores.view(-1, self.mlm_head.out_features), 
                                      labels.view(-1))
            loss = masked_lm_loss
            
        return {"loss": loss, "logits": prediction_scores} if loss is not None else prediction_scores


In [ ]:
"""
# Create model with configuration similar to distilbert-base-uncased
model = DistilBERT(
    vocab_size=tokenizer.vocab_size,
    hidden_size=768,
    num_layers=6,
    num_attention_heads=12,
    intermediate_size=3072,
    max_position_embeddings=512,
    dropout_rate=0.1
)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# Set up optimizer with weight decay
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

# Adjust batch size based on your GPU memory
batch_size = 8  # Start with a smaller batch size for Kaggle
train_dataloader = DataLoader(
    train_subset if 'train_subset' in locals() else masked_datasets["train"], 
    batch_size=batch_size, 
    shuffle=True
)

# Set up learning rate scheduler with warmup
num_epochs = 1  # Adjust based on your time constraints
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=int(0.1 * total_steps), 
    num_training_steps=total_steps
)

# Log model architecture to wandb
wandb.watch(model, log="all", log_freq=100)
"""

In [ ]:
"""
# Validation dataloader
validation_dataloader = DataLoader(
    masked_datasets["validation"], 
    batch_size=batch_size, 
    shuffle=False
)

def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs["loss"]
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Set up training loop
model.train()
global_step = 0
total_loss = 0
log_interval = 50  # Log more frequently on Kaggle
accumulation_steps = 4  # Accumulate gradients over multiple batches
save_interval = 5000  # Save model every 5000 steps

# Create directory for saving model checkpoints
os.makedirs("/kaggle/working/model_checkpoints", exist_ok=True)

for epoch in range(num_epochs):
    epoch_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for step, batch in enumerate(progress_bar):
        # Get batch and move to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs["loss"]

        # Scale loss for gradient accumulation
        loss = loss / accumulation_steps
        loss.backward()

        # Update weights every accumulation_steps batches
        if (step + 1) % accumulation_steps == 0:
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Update parameters
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            # Update metrics
            global_step += 1

            # Log to wandb
            if global_step % log_interval == 0:
                wandb.log({
                    "train/loss": loss.item() * accumulation_steps,  # Rescale loss
                    "train/learning_rate": scheduler.get_last_lr()[0],
                    "train/global_step": global_step
                })

                # Evaluate on validation set
                validation_loss = evaluate(model, validation_dataloader)
                wandb.log({
                    "val/loss": validation_loss
                })

            # Save checkpoint periodically
            if global_step % save_interval == 0:
                checkpoint_path = f"/kaggle/working/model_checkpoints/distilbert_step_{global_step}.pt"
                torch.save({
                    'epoch': epoch + 1,
                    'global_step': global_step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'loss': loss.item() * accumulation_steps,
                }, checkpoint_path)
                wandb.save(checkpoint_path)
                print(f"Checkpoint saved at step {global_step}")

        # Update progress bar
        epoch_loss += loss.item() * accumulation_steps
        progress_bar.set_postfix({
            "loss": loss.item() * accumulation_steps,
            "lr": scheduler.get_last_lr()[0]
        })

    # Log epoch metrics
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    wandb.log({
        "train/epoch": epoch + 1,
        "train/epoch_loss": avg_epoch_loss
    })

    # Save model after each epoch
    checkpoint_path = f"/kaggle/working/model_checkpoints/distilbert_epoch_{epoch+1}.pt"
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'loss': avg_epoch_loss,
    }, checkpoint_path)
    wandb.save(checkpoint_path)

    print(f"Epoch {epoch+1}/{num_epochs} completed. Average loss: {avg_epoch_loss:.4f}")

# Save final model with configuration
final_model_path = "/kaggle/working/distilbert_pretrained_final.pt"
torch.save({
    'model_state_dict': model.state_dict(),
    'vocab_size': tokenizer.vocab_size,
    'hidden_size': 768,
    'num_layers': 6,
    'num_attention_heads': 12,
    'intermediate_size': 3072,
    'max_position_embeddings': 512,
    'config': {
        'vocab_size': tokenizer.vocab_size,
        'hidden_size': 768,
        'num_layers': 6,
        'num_attention_heads': 12,
        'intermediate_size': 3072,
        'max_position_embeddings': 512,
        'dropout_rate': 0.1
    }
}, final_model_path)
wandb.save(final_model_path)

print(f"Pretraining completed! Model saved to {final_model_path}")
"""

In [ ]:
import torch
from torch import nn
from transformers import DistilBertConfig, DistilBertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset


In [ ]:
MODEL_CHECKPOINT = "/kaggle/working/fine_tuned_distilbert"
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [10]:
config = DistilBertConfig.from_pretrained(MODEL_CHECKPOINT, num_labels=2)

In [11]:
# Define model with classification head
class DistilBertForClassification(nn.Module):
    def __init__(self, config):
        super(DistilBertForClassification, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained(MODEL_CHECKPOINT, config=config)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.loss_fn = nn.CrossEntropyLoss()  # Add loss function

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])  # CLS token representation

        # Compute loss when labels are provided
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}


In [12]:
model = DistilBertForClassification(config)
model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertForClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
        

In [13]:
dataset = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [14]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [15]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)

In [17]:
trainer.train()

# Save the fine-tuned model
model_path = "./fine_tuned_distilbert"
trainer.save_model(model_path)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.286400,0.292394
2,0.163300,0.257574
3,0.071000,0.358047


In [ ]:
model.eval()
model.to(device)
example_text = "I loved the food."
inputs = tokenizer(example_text, return_tensors="pt").to(device)

In [24]:
with torch.no_grad():
    outputs = model(**inputs)

# Extract logits and get predicted class
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()

print(f"Predicted class: {predicted_class}")

Predicted class: 0
